<a href="https://colab.research.google.com/github/brontominds/bounce.demo/blob/master/Bounce_assignment_using_avg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from sklearn.utils import shuffle

In [0]:
data=pd.read_csv("train.csv")#Making it Relative Path -Mayank
data_ori=data.copy()

In [0]:
import matplotlib.pyplot as plt  # For plotting graphs 
from datetime import datetime    # To access datetime 
        
%matplotlib inline

In [0]:
def datetime(df):
  from datetime import datetime
  df['datetime'] = pd.to_datetime(df['datetime'])
  df['year'] = df['datetime'].dt.year 
  df['day'] = df['datetime'].dt.day 
  df['hour'] = df['datetime'].dt.hour
  return df

In [0]:
data_ori=datetime(data_ori)

In [0]:
def normalize(df,columns):
    result=df.copy()
    for feature_name in columns :
        mean_value = df[feature_name].mean()
        std_value = df[feature_name].std()
        result[feature_name] = (df[feature_name] - mean_value) / std_value
        
    return result

In [0]:
data_ori=normalize(data_ori,["temp","atemp","humidity","windspeed"])

In [30]:
data_ori.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,day,hour
0,2011-01-01 00:00:00,1,0,0,1,-1.333599,-1.092687,0.993167,-1.567682,3,13,16,2011,1,0
1,2011-01-01 01:00:00,1,0,0,1,-1.438841,-1.182367,0.941206,-1.567682,8,32,40,2011,1,1
2,2011-01-01 02:00:00,1,0,0,1,-1.438841,-1.182367,0.941206,-1.567682,5,27,32,2011,1,2
3,2011-01-01 03:00:00,1,0,0,1,-1.333599,-1.092687,0.681399,-1.567682,3,10,13,2011,1,3
4,2011-01-01 04:00:00,1,0,0,1,-1.333599,-1.092687,0.681399,-1.567682,0,1,1,2011,1,4


In [0]:
data_ori=data_ori[data_ori['day']==1]

In [32]:
data_ori.shape

(575, 15)

In [0]:
def dummy_data(data, columns):
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix=column)], axis=1)
        data = data.drop(column, axis=1)
    return data

In [0]:
data_ori = dummy_data(data_ori, ["season","weather","hour"])

In [0]:
data_ori=data_ori.drop('datetime',axis=1)

In [0]:
data=shuffle(data_ori,random_state=1)
nrows = len(data_ori)
training_validation_rows = int(nrows*0.8)
data=data_ori[0:training_validation_rows-1]
test1=data_ori[training_validation_rows:]

In [37]:
data.shape

(459, 42)

In [38]:
test1.shape

(115, 42)

In [0]:
def split_data(data):
    data_y = data["count"]
    
    

    data_x = data.drop(["count","casual","registered"], axis=1)

    

    return data_x.values, data_y

data_x, data_y = split_data(data)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.optimizers import Adam,SGD,RMSprop,Adagrad

In [0]:
adam = Adam(lr=1e-3, decay=1e-3 / 200)

In [0]:
from keras import backend
 
def rmse(y_true, y_pred):
	return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

In [46]:
from keras import regularizers
from keras.layers.core import Dropout
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = data_x.shape[1], activation='relu'))


# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))


# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='softplus'))

# Compile the network :
NN_model.compile(loss='mean_squared_logarithmic_error', optimizer=adam, metrics=['mean_absolute_error','mean_squared_error',rmse])
NN_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               5120      
_________________________________________________________________
dense_7 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_8 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_9 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 169,985
Trainable params: 169,985
Non-trainable params: 0
_________________________________________________________________


In [0]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose =1, save_best_only = True, mode ='auto')
early_stop=EarlyStopping(monitor='val_loss',patience=10)
callbacks_list = [checkpoint,early_stop]

In [48]:
history=NN_model.fit(data_x, data_y, epochs=100, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

Train on 367 samples, validate on 92 samples
Epoch 1/100
367/367 [==============================] - 0s 1ms/step - loss: 3.5881 - mean_absolute_error: 109.6928 - mean_squared_error: 24162.1356 - rmse: 109.6928 - val_loss: 1.7800 - val_mean_absolute_error: 167.5789 - val_mean_squared_error: 46728.0137 - val_rmse: 167.5789

Epoch 00001: val_loss improved from inf to 1.77996, saving model to Weights-001--1.77996.hdf5
Epoch 2/100
367/367 [==============================] - 0s 141us/step - loss: 2.0472 - mean_absolute_error: 99.5444 - mean_squared_error: 18781.3188 - rmse: 99.5444 - val_loss: 2.0140 - val_mean_absolute_error: 180.4978 - val_mean_squared_error: 57081.5190 - val_rmse: 180.4978

Epoch 00002: val_loss did not improve from 1.77996
Epoch 3/100
367/367 [==============================] - 0s 131us/step - loss: 1.9152 - mean_absolute_error: 103.5192 - mean_squared_error: 22459.1144 - rmse: 103.5192 - val_loss: 2.1605 - val_mean_absolute_error: 184.6250 - val_mean_squared_error: 60125.3

In [0]:
test1_x, test1_y = split_data(test1)

In [50]:
scores=NN_model.evaluate(test1_x,test1_y)

115/115 [==============================] - 0s 88us/step


In [51]:
print(NN_model.metrics_names)

['loss', 'mean_absolute_error', 'mean_squared_error', 'rmse']


In [0]:
test1_loss=scores[0]
test1_mae=scores[1]
test1_mse=scores[2]
test1_rmse=scores[3]

In [53]:
test1_mae

218.63342086128566